<a href="https://colab.research.google.com/github/louispaulet/Data_dependencies_and_data_integration_course/blob/main/DDDI_Project_IND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas-profiling

In [ ]:
import pandas as pd
import io

df = pd.read_csv('Table1.csv')
df


,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


In [ ]:
df.describe()

,A,B,C,D,E
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,1.142857,1.428571,0.571429,1.714286,1.142857
std,1.951800,1.272418,0.534522,0.487950,1.573592
min,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.500000,0.000000,1.500000,0.000000
50%,0.000000,1.000000,1.000000,2.000000,0.000000
75%,2.000000,2.500000,1.000000,2.000000,2.000000
max,4.000000,3.000000,1.000000,2.000000,4.000000


We create a df2 with a "test_col" column that will include all other columns.

In [ ]:

df3 = df.copy()
df3['test_col'] = range(0, len(df))
df3 = df3.drop('E', 1)
df3

,A,B,C,D,test_col
0,0,0,0,2,0
1,0,1,0,2,1
2,0,2,0,2,2
3,0,3,1,2,3
4,4,1,1,1,4
5,4,3,1,1,5
6,0,0,1,2,6


For the naive solution, we generate the search space as follows :
all permutations of df columns against all permutations of df columns.

In [ ]:
from itertools import permutations

column_combination = []
for i in range(1, len(df.columns)+1):
  column_combination.append(list(permutations(df.columns, i)))

instructions = []
for gen in column_combination:
  for permLeft in gen:
    for permRight in gen:
      #filter out trivial inclusions like A in A and AB in AB, etc...
      if (permLeft != permRight):
        instructions.append((permLeft, permRight))


instructions

[(('A',), ('B',)),
 (('A',), ('C',)),
 (('A',), ('D',)),
 (('A',), ('E',)),
 (('B',), ('A',)),
 (('B',), ('C',)),
 (('B',), ('D',)),
 (('B',), ('E',)),
 (('C',), ('A',)),
 (('C',), ('B',)),
 (('C',), ('D',)),
 (('C',), ('E',)),
 (('D',), ('A',)),
 (('D',), ('B',)),
 (('D',), ('C',)),
 (('D',), ('E',)),
 (('E',), ('A',)),
 (('E',), ('B',)),
 (('E',), ('C',)),
 (('E',), ('D',)),
 (('A', 'B'), ('A', 'C')),
 (('A', 'B'), ('A', 'D')),
 (('A', 'B'), ('A', 'E')),
 (('A', 'B'), ('B', 'A')),
 (('A', 'B'), ('B', 'C')),
 (('A', 'B'), ('B', 'D')),
 (('A', 'B'), ('B', 'E')),
 (('A', 'B'), ('C', 'A')),
 (('A', 'B'), ('C', 'B')),
 (('A', 'B'), ('C', 'D')),
 (('A', 'B'), ('C', 'E')),
 (('A', 'B'), ('D', 'A')),
 (('A', 'B'), ('D', 'B')),
 (('A', 'B'), ('D', 'C')),
 (('A', 'B'), ('D', 'E')),
 (('A', 'B'), ('E', 'A')),
 (('A', 'B'), ('E', 'B')),
 (('A', 'B'), ('E', 'C')),
 (('A', 'B'), ('E', 'D')),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C'), ('A', 'D')),
 (('A', 'C'), ('A', 'E')),
 (('A', 'C'), ('B', 'A')),

We can see that the search space for 5 columns is 32 500 combinations (32 825 combinations if we don't remove the trivial inclusions) of column inclusions.

In [ ]:
len(instructions)

32500

For concatenating all the columns into easily comparable tuples, we convert the dataframe values to string type :

In [ ]:
df = df.applymap(str)
df

,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


A test of string concat between two columns :

In [ ]:
df2 = df.copy()

df[['A', 'B']].agg(''.join, axis=1)

0    00
1    01
2    02
3    03
4    41
5    43
6    00
dtype: object

In [ ]:
def concat_and_remove_duplicates(df, instruction):
  return df[list(instruction)].agg(''.join, axis=1).drop_duplicates()

concat_and_remove_duplicates(df, ('A', 'B', 'C', 'D'))

0    0002
1    0102
2    0202
3    0312
4    4111
5    4311
6    0012
dtype: object

In [ ]:
#we then check how inclusions work :

test_df = concat_and_remove_duplicates(df, ('A', 'B', 'C', 'D'))

#0002 is included, 0003 is not. we want to negate these booleans to exclude exclusions(and get a list of exclusions)
print(not('0002' in test_df.values), not('0003' in test_df.values))

False True


For checking inclusion, we will concatenate all strings of a column combination, remove duplicates in resulting dataframes (the two dataframes created from the 2 combinations to check), and check for inclusion from the left df to the right df.

In [ ]:
#test with 100 first instructions
instructions2 = instructions[:100]
list_of_exclusions = []

for instruction in instructions2:
  leftDF = concat_and_remove_duplicates(df, instruction[0])
  rightDF = concat_and_remove_duplicates(df, instruction[1])
  for leftRow in leftDF:
    if (not(leftRow in rightDF.values)):
      list_of_exclusions.append(instruction)
      break


# set(instructions2)
set(instructions2) - set(list_of_exclusions)

{(('A',), ('E',)),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C'), ('E', 'B')),
 (('A', 'C'), ('E', 'C')),
 (('A', 'D'), ('A', 'B')),
 (('A', 'D'), ('E', 'D')),
 (('C',), ('B',)),
 (('D',), ('B',))}

We can then sum up all this code into a single IND (inclusion depedency) function :

In [ ]:
def ind(df):
  df = df.applymap(str)
  column_combination = []
  for i in range(1, len(df.columns)+1):
    column_combination.append(list(permutations(df.columns, i)))

  instructions = []
  for gen in column_combination:
    for permLeft in gen:
      for permRight in gen:
        #filter out trivial inclusions like A in A and AB in AB, etc...
        if (permLeft != permRight):
          instructions.append((permLeft, permRight))

  list_of_exclusions = []

  cpt = 0
  for instruction in instructions:
    leftDF = df[list(instruction[0])].agg(''.join, axis=1).drop_duplicates()
    rightDF = df[list(instruction[1])].agg(''.join, axis=1).drop_duplicates()
    for leftRow in leftDF:
      if (not(leftRow in rightDF.values)):
        list_of_exclusions.append(instruction)
        break
    cpt+=1
    if (cpt % 1000 == 0):
      print(str(cpt) +" / "+str(len(instructions)))

  #return all inclusions except the exclusions
  return set(instructions) - set(list_of_exclusions)

We run the function on the default dataset (>2 mins) : 

In [ ]:
ind(df)

1000 / 32500
2000 / 32500
3000 / 32500
4000 / 32500
5000 / 32500
6000 / 32500
7000 / 32500
8000 / 32500
9000 / 32500
10000 / 32500
11000 / 32500
12000 / 32500
13000 / 32500
14000 / 32500
15000 / 32500
16000 / 32500
17000 / 32500
18000 / 32500
19000 / 32500
20000 / 32500
21000 / 32500
22000 / 32500
23000 / 32500
24000 / 32500
25000 / 32500
26000 / 32500
27000 / 32500
28000 / 32500
29000 / 32500
30000 / 32500
31000 / 32500
32000 / 32500


{(('A',), ('E',)),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C'), ('E', 'B')),
 (('A', 'C'), ('E', 'C')),
 (('A', 'C', 'D'), ('A', 'B', 'D')),
 (('A', 'C', 'D'), ('E', 'B', 'D')),
 (('A', 'C', 'D'), ('E', 'C', 'D')),
 (('A', 'D'), ('A', 'B')),
 (('A', 'D'), ('E', 'D')),
 (('A', 'D', 'C'), ('A', 'D', 'B')),
 (('A', 'D', 'C'), ('E', 'D', 'B')),
 (('A', 'D', 'C'), ('E', 'D', 'C')),
 (('C',), ('B',)),
 (('C', 'A'), ('B', 'A')),
 (('C', 'A'), ('B', 'E')),
 (('C', 'A'), ('C', 'E')),
 (('C', 'A', 'D'), ('B', 'A', 'D')),
 (('C', 'A', 'D'), ('B', 'E', 'D')),
 (('C', 'A', 'D'), ('C', 'E', 'D')),
 (('C', 'D'), ('B', 'D')),
 (('C', 'D', 'A'), ('B', 'D', 'A')),
 (('C', 'D', 'A'), ('B', 'D', 'E')),
 (('C', 'D', 'A'), ('C', 'D', 'E')),
 (('D',), ('B',)),
 (('D', 'A'), ('B', 'A')),
 (('D', 'A'), ('D', 'E')),
 (('D', 'A', 'C'), ('D', 'A', 'B')),
 (('D', 'A', 'C'), ('D', 'E', 'B')),
 (('D', 'A', 'C'), ('D', 'E', 'C')),
 (('D', 'C'), ('D', 'B')),
 (('D', 'C', 'A'), ('D', 'B', 'A')),
 (('D', 'C', 'A'), ('D', 'B

We run the function on df2 to check how inclusions changes when we have a column that contains all values :

In [ ]:
ind(df3)

1000 / 32500
2000 / 32500
3000 / 32500
4000 / 32500
5000 / 32500
6000 / 32500
7000 / 32500
8000 / 32500
9000 / 32500
10000 / 32500
11000 / 32500
12000 / 32500
13000 / 32500
14000 / 32500
15000 / 32500
16000 / 32500
17000 / 32500
18000 / 32500
19000 / 32500
20000 / 32500
21000 / 32500
22000 / 32500
23000 / 32500
24000 / 32500
25000 / 32500
26000 / 32500
27000 / 32500
28000 / 32500
29000 / 32500
30000 / 32500
31000 / 32500
32000 / 32500


{(('A',), ('test_col',)),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C', 'D'), ('A', 'B', 'D')),
 (('A', 'D'), ('A', 'B')),
 (('A', 'D'), ('test_col', 'D')),
 (('A', 'D', 'C'), ('A', 'D', 'B')),
 (('B',), ('test_col',)),
 (('C',), ('B',)),
 (('C',), ('test_col',)),
 (('C', 'A'), ('B', 'A')),
 (('C', 'A', 'D'), ('B', 'A', 'D')),
 (('C', 'D'), ('B', 'D')),
 (('C', 'D', 'A'), ('B', 'D', 'A')),
 (('D',), ('B',)),
 (('D',), ('test_col',)),
 (('D', 'A'), ('B', 'A')),
 (('D', 'A'), ('D', 'test_col')),
 (('D', 'A', 'C'), ('D', 'A', 'B')),
 (('D', 'C'), ('D', 'B')),
 (('D', 'C', 'A'), ('D', 'B', 'A'))}

We find that all single columns are included in the test column and even find supersets of the test_column